<a href="https://colab.research.google.com/github/lolomarka/web-analysis/blob/main/web_anal_prakt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
!pip3 install requests urllib3 warcio pandas beautifulsoup4

In [114]:
import requests
import json
import os
import pandas as pd
from urllib.parse import quote_plus, unquote_plus
from requests.utils import requote_uri

from warcio.archiveiterator import ArchiveIterator

In [115]:
def search_url(index_name, url):
  encoded_url = quote_plus(url)
  index_url = f'http://index.commoncrawl.org/{index_name}-index?url={encoded_url}&output=json'
  response = requests.get(index_url)

  if response.status_code == 200:
    records = response.text.strip().split('\n')
    return [json.loads(record) for record in records]
  else:
    return None

def distinct_by_url(not_unique_records_list):
  unique = {}
  for result in not_unique_records_list:
    if result['url'] not in unique:
      unique[result['url']] = result
  return unique


In [116]:
index_names = { 'CC-MAIN-2024-38', 'CC-MAIN-2024-33', 'CC-MAIN-2024-30', 'CC-MAIN-2024-26', 'CC-MAIN-2024-22', 'CC-MAIN-2024-18', 'CC-MAIN-2024-10', 'CC-MAIN-2024-50' }

# Для повышения шансов что=то найти, ищем на русской википедии.
url_for_search = 'ru.wikipedia.org/*'

results = []

for index in index_names:
  s_result = search_url(index, url_for_search)
  if s_result:
    results += s_result

len(results)

93396

In [117]:
unique_results = distinct_by_url(results)
len(unique_results)

91209

In [118]:
def fetch_single_record(warc_record_filename, offset, length):
    s3_url = f'https://data.commoncrawl.org/{warc_record_filename}'

    byte_range = f'bytes={offset}-{offset + length - 1}'

    response = requests.get(
        s3_url,
        headers={'Range': byte_range},
        stream=True
    )

    if response.status_code == 206:
        stream = ArchiveIterator(response.raw)
        for warc_record in stream:
            if warc_record.rec_type == 'response':
                return warc_record.content_stream().read()
    else:
        print(f"Failed to fetch data: {response.status_code}")

    return None

In [119]:
keywords = [ 'Перм', 'Пастернак', 'ПНИПУ', 'МФТИ', 'МГУ', 'Московский государственный университет', 'ИТАС', 'Пермский Национальный Исследовательский Политехнический Университет']
keywords

['Перм',
 'Пастернак',
 'ПНИПУ',
 'МФТИ',
 'МГУ',
 'Московский государственный университет',
 'ИТАС',
 'Пермский Национальный Исследовательский Политехнический Университет']

In [120]:
filtered_results = []
for result in unique_results.values():
	if any(requote_uri(keyword) in result['url'] for keyword in keywords):
		filtered_results.append(result)

len(filtered_results)

21

In [121]:
html_results = {}

for result in filtered_results:
	record = fetch_single_record(result['filename'], int(result['offset']), int(result['length']))
	if record:
		html_results[result['url']] = record



In [123]:
from bs4 import BeautifulSoup

for url, html in html_results.items():
  beautiful_soup = BeautifulSoup(html, 'html.parser')
  print(f'Название статьи - {beautiful_soup.title.string}; URL - {unquote_plus(url)}')
  for keyword in keywords:
    if keyword.casefold() in beautiful_soup.get_text().casefold():
      print(f"Ключевое слово: {keyword} в статье {beautiful_soup.title.string}")
  print("\n")



Название статьи - Барда (Пермский край) — Википедия; URL - https://ru.wikipedia.org/wiki/Барда_(Пермский_край)
Ключевое слово: Перм в статье Барда (Пермский край) — Википедия


Название статьи - Башня смерти (Пермь) — Википедия; URL - https://ru.wikipedia.org/wiki/Башня_смерти_(Пермь)
Ключевое слово: Перм в статье Башня смерти (Пермь) — Википедия


Название статьи - Губернатор Пермского края — Википедия; URL - https://ru.wikipedia.org/wiki/Губернатор_Пермского_края
Ключевое слово: Перм в статье Губернатор Пермского края — Википедия


Название статьи - Ельники (Пермский край) — Википедия; URL - https://ru.wikipedia.org/wiki/Ельники_(Пермский_край)
Ключевое слово: Перм в статье Ельники (Пермский край) — Википедия


Название статьи - Звёздный (Пермский край) — Википедия; URL - https://ru.wikipedia.org/wiki/Звёздный_(Пермский_край)
Ключевое слово: Перм в статье Звёздный (Пермский край) — Википедия


Название статьи - Административно-территориальное деление Пермского края — Википедия; URL -